In [ ]:
-- Checking for the Month End Date. 
-- Only proceed if there are 
--      a) rows returned
--      b) we havent processed the returned date yet.
-- Store the date as it will be used for a filter downstream

with
billing_periods as (
    select * from staging.salesforce.stg_salesforce__billing_period_c
)

select
    id,
    name,
    status_c,
    month_start_date_c,
    month_end_date_c, -- critical for determining when
    period_end_date_c
from billing_periods
where status_c = 'Closed'
  and period_end_date_c >= date_trunc('month', CURRENT_DATE)


# Use the Below Query
#### This query will return the Net Balances for the last closed period.  All the results will be selectively pushed to Flowpoint.
#### The Integration process contains the a list of what was already pushed to Flowpoint.  If the results were already pushed, then that Opportunity will be skipped for the next execution.

In [ ]:
-- get the net account balances here
--
-- This process will always return rows.
--   It's up to the integration to ensure that it only calls the api if/when the traacting 
--   document says that it needs to be updated.
-- 
-- The process runs every day.
--      - run this query.
--      - compare each row to last updated row in the integration
--      - if row is old:
--          - call the api
--              - Success: update tracking document
--              - Failure: send notification and move on
--      - if row is current
--          - ignore and move on
--       
--  Sample tracking document
-- { "accounts": [
--     {
--         "account_id": "xyz",
--         "account_name": "some name",
--         "opportunities": [
--             {
--                 "opportunity_id": "xyzabc",
--                 "opportunity_name": "some name",
--                 "last_flowpoint_update": "YYYY-MM-DD"
--             },
--             {
--                 "opportunity_id": "xyzabc",
--                 "opportunity_name": "some name",
--                 "last_flowpoint_update": "YYYY-MM-DD"
--             }
--         ]
--     },
--     {
--         "account_id": "abc",
--         "account_name": "some name",
--         "opportunities": [
--             {
--                 "opportunity_id": "abcxyz",
--                 "opportunity_name": "some name",
--                 "last_flowpoint_update": "YYYY-MM-DD"
--             },
--             {
--                 "opportunity_id": "xyzabc",
--                 "opportunity_name": "some name",
--                 "last_flowpoint_update": "YYYY-MM-DD"
--             }
--         ]
--     },    
--     ]
-- }

with
get_month as (
    select max(month_end_date_c) as month_end_date
    from staging.salesforce.stg_salesforce__billing_period_c 
    where status_c = 'Closed'
),

loans as (
    select * from staging.intermediate.int_salesforce__loan
),

ledgers as (
    select * from staging.intermediate.int_salesforce__transaction_ledger
),

accounts as (
    select 
        id as account_id,
        name as account_name
    from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

opportunities as (
    select 
        id as opportunity_id,
        name as opportunity_name,
        account_id
    from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

ledgers_filtered as (
    select 
        *
    from ledgers
    where transaction_date_c<=(select month_end_date from get_month)
      and is_deleted=false
),

advances as (
    select
        l.loan_c, 
        p.opportunity_c as opportunity_id,
        sum(coalesce(l.transaction_amount_c,0)) as total_advanced_amount
    from ledgers_filtered l
    inner join loans p 
        on p.id=l.loan_c
    where p.loan_status_c='Open - Balance Remaining'
      and l.transaction_type_c='Principal'
      and l.line_item_name_c not ilike 'Interest%'
      and l.line_item_name_c in ('Principal Advanced', 'Rollover Advance')
    group by 
        l.loan_c, p.opportunity_c
),

repayments as (
    select 
        l.loan_c,
        p.opportunity_c as opportunity_id,
        sum(coalesce(l.transaction_amount_c,0)) as total_repaid_amount
    from ledgers_filtered l
    inner join loans p on p.id=l.loan_c
    where p.loan_status_c='Open - Balance Remaining'
      and l.transaction_type_c='Principal'
      and l.line_item_name_c not like ('Interest%')
      and l.line_item_name_c in ('Principal Repaid', 'Rollover Repayment')
    group by 
        l.loan_c, p.opportunity_c
),

net_transaction_amount as (
    select
        a.loan_c as loan,
        a.opportunity_id,
        coalesce(a.total_advanced_amount, 0) - coalesce(r.total_repaid_amount, 0) as net_transaction_amount,
        a.total_advanced_amount,
        r.total_repaid_amount
    from advances a
    left join repayments r
        on a.loan_c = r.loan_c
),

final as (
    select
        m.month_end_date,
        p.loan,
        a.account_id,
        a.account_name,
        p.opportunity_id,
        o.opportunity_name,
        p.net_transaction_amount
        -- p.total_advanced_amount,
        -- p.total_repaid_amount
    from net_transaction_amount p
    inner join opportunities o
        on o.opportunity_id=p.opportunity_id
    inner join accounts a 
        on a.account_id=o.account_id
    cross join get_month m
)

select * from final 
--where opportunity_id in ('0066T000017JRXxQAO', '0066T000017JAaQQAW')
order by account_id, opportunity_id
    
-- Still need to identify the oppoetunity ids and maybe account ids here

### The following is the query to retrieve the latest 'Closed' period.

In [ ]:
with
get_month as (
    select 
        max(month_end_date_c) as month_end_date
    from staging.salesforce.stg_salesforce__billing_period_c 
    where status_c = 'Closed'
),

loans as (
    select * from staging.intermediate.int_salesforce__loan
),

ledgers as (
    select * from staging.intermediate.int_salesforce__transaction_ledger
),

accounts as (
    select 
        id as account_id,
        name as account_name
    from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

opportunities as (
    select 
        id as opportunity_id,
        name as opportunity_name,
        account_id
    from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

ledgers_filtered as (
    select 
        *
    from ledgers
    where transaction_date_c<=(select month_end_date from get_month)
      and is_deleted=false
),

advances as (
    select
        l.loan_c, 
        p.opportunity_c as opportunity_id,
        sum(coalesce(l.transaction_amount_c,0)) as total_advanced_amount
    from ledgers_filtered l
    inner join loans p on p.id=l.loan_c
    -- the following is used to calculate the net balances - this is for total advances
    where p.loan_status_c='Open - Balance Remaining'
      and l.transaction_type_c='Principal'
      and l.line_item_name_c in ('Principal Advanced', 'Rollover Advance')
    group by 
        l.loan_c, p.opportunity_c
),

repayments as (
    select 
        l.loan_c,
        p.opportunity_c as opportunity_id,
        sum(coalesce(l.transaction_amount_c,0)) as total_repaid_amount
    from ledgers_filtered l
    inner join loans p on p.id=l.loan_c
    -- the following is used to calculate the net balances - this is for total repayments
    where p.loan_status_c='Open - Balance Remaining'
      and l.transaction_type_c='Principal'
      and l.line_item_name_c in ('Principal Repaid', 'Rollover Repayment')
    group by 
        l.loan_c, p.opportunity_c
),

net_transaction_amount as (
    select
        a.loan_c as loan,
        a.opportunity_id,
        coalesce(a.total_advanced_amount, 0) - coalesce(r.total_repaid_amount, 0) as net_transaction_amount,
        a.total_advanced_amount,
        r.total_repaid_amount
    from advances a
    left join repayments r
        on a.loan_c = r.loan_c
),

final as (
    select
        m.month_end_date,
        p.loan,
        a.account_id,
        a.account_name,
        p.opportunity_id,
        o.opportunity_name,
        p.net_transaction_amount,
        p.total_advanced_amount,
        p.total_repaid_amount
    from net_transaction_amount p
    inner join opportunities o
        on o.opportunity_id=p.opportunity_id
    inner join accounts a 
        on a.account_id=o.account_id
    cross join get_month m
)

-- select * from get_month

select * from final 
order by account_name

    